In [17]:
!pip install qiskit-machine-learning==0.6.1

In [37]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
import qiskit
from cuquantum import *
import time
#import nvtx

In [38]:
traindata = pd.read_csv('../../Pika/NSRRC/fashion-mnist_train.csv')
filt = (traindata['label'] < 2)
two_class = traindata.loc[filt]

In [39]:
def fashion_train_data(n_dim = 2,datasize = 2):
    data_train  = two_class.iloc[:,1:785] / 255.0
    label_train = pd.DataFrame([two_class.iloc[:,0]]).T
    l_train=pd.DataFrame([two_class.iloc[:,0]]).T
    X_train, X_val, Y_train, Y_val = train_test_split(data_train, l_train, test_size = 0.25, random_state=255)
    X_train = StandardScaler().fit_transform(X_train)
    X_val   = StandardScaler().fit_transform(X_val)
    pca = PCA(n_components=n_dim).fit(X_train)
    X_train = pca.transform(X_train)
    X_train = X_train[:datasize]
    Y_train = np.array(Y_train).ravel()
    Y_train = Y_train[:datasize]  
    return X_train, Y_train

In [40]:
def qc_qiskit2cuquantum(x_t1, x_t2, n_dim):
    zz_map = ZZFeatureMap(feature_dimension=n_dim, reps=1, entanglement="linear", insert_barriers=True)
    zz_kernel = QuantumKernel(feature_map=zz_map)  
    zz_circuit = zz_kernel.construct_circuit(x_t1,x_t2,measurement=False,is_statevector_sim=False)
    converter = CircuitToEinsum(zz_circuit, backend='cupy')
    a = str(0).zfill(n_dim)
    exp, oper = converter.amplitude(a)  
    return zz_circuit, exp, oper

In [41]:
import cupy as cp

def get_operands(mode, datasize, x_t, n_dim):
    OPER = []
    for i1 in range(2,datasize+1):
        for i2 in range(1,i1):
            _, _, oper = qc_qiskit2cuquantum(x_t[i1-1],x_t[i2-1],n_dim)
            if mode == 'gpu':    # Data transfer from host to GPU
                oper_gpu = [cp.asarray(tensor) for tensor in oper]
                OPER.append(oper_gpu)
            elif mode == 'cpu':
                OPER.append(oper)

    return OPER

In [42]:
def get_kernel_matrix_0(x_t,datasize,n_dim):
    kernel_matrix = np.zeros((datasize,datasize))

    qc, exp, oper = qc_qiskit2cuquantum(x_t[0], x_t[0], n_dim)
    path, info = contract_path(exp,*oper)

    t0 = time.time()
    for i1 in range(2,datasize+1):
        for i2 in range(1,i1):
            qc, exp_tmp, oper_tmp = qc_qiskit2cuquantum(x_t[i1-1],x_t[i2-1],n_dim)
            amp = contract(exp_tmp, *oper_tmp, optimize = {'path' : path})
            kernel_matrix[i1-1][i2-1] = np.round(np.sqrt(np.square(amp.real)+np.square(amp.imag)),5)
    kernel_matrix = kernel_matrix+kernel_matrix.T+np.diag(np.ones((datasize)))
    print("Time = ", time.time()-t0)
    
    return kernel_matrix 

In [43]:
n_dim, datasize  = 50, 100
x_t, y_t = fashion_train_data(n_dim, datasize)

In [44]:
""" Original (qc_qiskit2cuquantum included) """

get_kernel_matrix_0(x_t, datasize, n_dim)

Time =  1265.3554968833923


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [45]:
def get_kernel_matrix_cpu(OPER, x_t,datasize,n_dim):
    kernel_matrix = np.zeros((datasize,datasize))

    _, exp, _ = qc_qiskit2cuquantum(x_t[0], x_t[0], n_dim)
    oper, idx = OPER[0], -1
    path, info = contract_path(exp,*oper)
    
    t0 = time.time()
    for i1 in range(2,datasize+1):
        for i2 in range(1,i1):
            idx += 1
            amp = contract(exp, *OPER[idx], optimize = {'path' : path})
            kernel_matrix[i1-1][i2-1] = np.round(np.sqrt(np.square(amp.real)+np.square(amp.imag)),5)
    kernel_matrix = kernel_matrix + kernel_matrix.T + np.diag(np.ones((datasize)))
    print("Time = ", time.time()-t0)
    
    return kernel_matrix 

In [ ]:
""" Original (qc_qiskit2cuquantum excluded) """

OPER = get_operands('cpu', datasize, x_t, n_dim)
get_kernel_matrix_cpu(OPER, x_t, datasize, n_dim)

In [ ]:
def get_kernel_matrix_gpu(OPER, x_t,datasize,n_dim):
    kernel_matrix = cp.zeros((datasize,datasize))

    _, exp, _ = qc_qiskit2cuquantum(x_t[0], x_t[0], n_dim)
    oper, idx = OPER[0], -1
    path, info = contract_path(exp,*oper)
    n = NetworkOptions(blocking="auto")
    
    t0 = time.time()
    for i1 in range(2,datasize+1):
        for i2 in range(1,i1):
            idx += 1
            amp = contract(exp, *OPER[idx], optimize = {'path' : path}, options=n)
            kernel_matrix[i1-1][i2-1] = cp.round(cp.sqrt(cp.square(amp.real)+cp.square(amp.imag)),5)
    kernel_matrix = kernel_matrix + kernel_matrix.T+cp.diag(cp.ones((datasize)))
    print("Time = ", time.time()-t0)
    
    return kernel_matrix 

In [ ]:
""" TNs are on GPU, contract with blocking="auto" """

OPER = get_operands('gpu', datasize, x_t, n_dim)
get_kernel_matrix_gpu(OPER, x_t, datasize, n_dim)

In [ ]:
def get_kernel_matrix_gpu1(OPER, x_t,datasize,n_dim):
    kernel_matrix = cp.zeros((datasize,datasize))

    _, exp, _ = qc_qiskit2cuquantum(x_t[0], x_t[0], n_dim)
    oper, idx = OPER[0], -1
    with Network(exp, *oper) as tn:
        path, info = tn.contract_path()
        t0 = time.time()
        for i1 in range(2,datasize+1):
            for i2 in range(1,i1):
                idx += 1
                tn.reset_operands(*OPER[idx])
                amp = tn.contract()
                kernel_matrix[i1-1][i2-1] = cp.round(cp.sqrt(cp.square(amp.real)+cp.square(amp.imag)),5)
        kernel_matrix = kernel_matrix + kernel_matrix.T+cp.diag(cp.ones((datasize)))
        print("Time = ", time.time()-t0)
    
    return kernel_matrix 

In [ ]:
""" TNs are on GPU, contract with cuquantum.Network specified (https://docs.nvidia.com/cuda/cuquantum/latest/python/api/generated/cuquantum.Network.html) """

#OPER = get_operands('gpu', datasize, x_t, n_dim)
get_kernel_matrix_gpu1(OPER, x_t, datasize, n_dim)